# IMPORTAR UNA BASE DE DATOS DE PGADMIN 4

¿No les ha pasado que encuentran mil ejemplos para conectar Python con SQL Server en Windows, pero casi nadie explica cómo hacerlo fuera del ecosistema Microsoft? 🤯

Pues bien — también podemos trabajar sin problemas desde macOS, usando motores alternativos como PostgreSQL (PGAdmin 4) y conectando la base directamente a Python para análisis, modelos y visualización. 🐍💾
La clave está en usar librerías como psycopg o SQLAlchemy, lo que nos permite consultar, transformar y explotar los datos sin depender de Windows.

Explorar otros entornos abre más posibilidades, más libertad y más aprendizaje.
La data es la misma — lo poderoso es cómo la conectas 🔥

## 1er paso es importar las librerias que vamos a usar

In [1]:
import pandas as pd
import psycopg

## 2do paso establcer la conexión con la base de datos 

In [2]:
conn = psycopg.connect(
    host="localhost", #Según el nombre de tu host, esto lo vez en el entorno de PGAdmin 4
    port=5432, 
    user="USUARIO",      # Cambia por tu usuario
    password="CONTRASEÑA", # Colocas tu contraseña
    dbname="BASE_COMPLETA"      # Cambia por el nombre de tu base
)

cur = conn.cursor()
cur.execute("SELECT version();")
print(cur.fetchone())

('PostgreSQL 16.1 on x86_64-apple-darwin20.6.0, compiled by Apple clang version 12.0.5 (clang-1205.0.22.9), 64-bit',)


## 3er paso colocamos la consulta SQL para importar los datos que se van a usar

In [3]:
## Usamos algunas columnas de una base de datos de colaboradores de una empresa X
ACTIVOS_FEB_24 = pd.read_sql(
    """
    SELECT 
        "periodo",
        "UNIDAD DE NEGOCIO",
        "pais",
        "RAZON SOCIAL / PLANILLA",
        "gerencia",
        "area",
        "POSICION / PUESTO / CARGO",
        "DOCUMENTO IDENTIDAD / CEDULA / RUT",
        "FECHA DE NACIMIENTO (DD/MM/YYYY)",
        "GENERO (F/M)",
        "FECHA DE INGRESO (DD/MM/YYYY)"
    FROM "DATOS_COMPILADOS_FEB24"
    WHERE "FECHA DE CESE (DD/MM/YYYY)" IS NULL;
    """,
    conn
)

/var/folders/7l/8rl5yq1j1g9684gbgh80kqq80000gn/T/ipykernel_63144/2954168160.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ACTIVOS_FEB_24 = pd.read_sql(


### 3.1 Visualizamos la información de las columnas que hemos importado

In [5]:
ACTIVOS_FEB_24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28298 entries, 0 to 28297
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   periodo                             28298 non-null  object
 1   UNIDAD DE NEGOCIO                   28298 non-null  object
 2   pais                                28298 non-null  object
 3   RAZON SOCIAL / PLANILLA             28298 non-null  object
 4   gerencia                            28290 non-null  object
 5   area                                28006 non-null  object
 6   POSICION / PUESTO / CARGO           28298 non-null  object
 7   DOCUMENTO IDENTIDAD / CEDULA / RUT  24166 non-null  object
 8   FECHA DE NACIMIENTO (DD/MM/YYYY)    28198 non-null  object
 9   GENERO (F/M)                        28298 non-null  object
 10  FECHA DE INGRESO (DD/MM/YYYY)       28202 non-null  object
dtypes: object(11)
memory usage: 2.4+ MB


### 3.2 Visualizamos la descripción de nuestros datos importados

In [4]:
ACTIVOS_FEB_24.describe()

,periodo,UNIDAD DE NEGOCIO,pais,RAZON SOCIAL / PLANILLA,gerencia,area,POSICION / PUESTO / CARGO,DOCUMENTO IDENTIDAD / CEDULA / RUT,FECHA DE NACIMIENTO (DD/MM/YYYY),GENERO (F/M),FECHA DE INGRESO (DD/MM/YYYY)
count,28298,28298,28298,28298,28290,28006,28298,24166,28198,28298,28202
unique,2,7,9,33,52,218,1117,6176,5352,2,1876
top,2024-02-01,CARIBE,Rep. Dom.,ISM Comercial RD,GERENCIA COMERCIAL,JEFATURA DE DISTRIBUCION,EJECUTIVO DE VENTAS,0,1991-03-17,M,2023-06-01
freq,14380,14000,12376,8950,7362,4642,3342,96,30,24428,3278


### 3.3 Visualizamos si tenemos valores nulos

In [10]:
ACTIVOS_FEB_24.isnull().sum()

periodo                                  0
UNIDAD DE NEGOCIO                        0
pais                                     0
RAZON SOCIAL / PLANILLA                  0
gerencia                                 8
area                                   292
POSICION / PUESTO / CARGO                0
DOCUMENTO IDENTIDAD / CEDULA / RUT    4132
FECHA DE NACIMIENTO (DD/MM/YYYY)       100
GENERO (F/M)                             0
FECHA DE INGRESO (DD/MM/YYYY)           96
dtype: int64

### 3.4 Visualizamos si tenemos valores duplicados

In [11]:
duplicados_por_unidad = (
    ACTIVOS_FEB_24
    .groupby(["UNIDAD DE NEGOCIO", "DOCUMENTO IDENTIDAD / CEDULA / RUT"])
    .size()
    .reset_index(name="repeticiones")
    .query("repeticiones > 1")       # filtra solo duplicados reales
    .groupby("UNIDAD DE NEGOCIO")     # vuelve a agrupar por unidad
    .size()
    .reset_index(name="total_documentos_duplicados")
)

duplicados_por_unidad

,UNIDAD DE NEGOCIO,total_documentos_duplicados
0,AGRO ISM,52
1,BRASIL,628
2,CARIBE,2646
3,CORPORATIVO,65
4,GT,320
5,NUEVOS NEGOCIOS,38
6,PERU-CHILE,2428


### Con estos scripts verificamos correctamente la importación de nuestra base desde PGAdmin4, evaluando además la calidad de los datos: identificamos si existen valores nulos por columna y si hay registros duplicados en campos que deberían ser únicos.

In [12]:
!jupyter nbconvert --to html "Conexión PGAdmin 4.ipynb"

[NbConvertApp] Converting notebook Conexión PGAdmin 4.ipynb to html
[NbConvertApp] Writing 291181 bytes to Conexión PGAdmin 4.html
